# HW 5

In [1]:
from __future__ import division
from __future__ import print_function
import serial
from math import atan2, acos, sqrt, pi, cos, sin
from math import radians as deg2rad
from math import degrees as rad2deg

# Inverse Code

Below is my class I made for the arm. They do not have to do a class. I also had to comment out the serial commands ... just ignore them.

<img src="cos-law.png" width="50%">

In [6]:
def cosine_law(a, b, c, phase=False):
    """
    c - opposite side of angle wanted
    a,b - sides next to angle wanted
    angle - radians
    """
    if phase:
        angle = acos((c**2 - (a**2 + b**2))/(2*a*b))
    else:
        angle = acos((c**2 - (a**2 + b**2))/(-2*a*b))

    return angle

In [9]:
def mag(a, b, c=0):
    return sqrt(a**2 + b**2 + c**2)


class Arm(object):
    CLAW_OPEN = 0
    CLAW_GRAB = 90
    CLAW_CLOSED = 180

    def __init__(self, port, pwm=[800, 2300]):
        self.pwm = pwm

        # calculate angle2pwm coefficients
        servo_range = [0, 180]
        self.slope = (self.pwm[1] - self.pwm[0])/(servo_range[1] - servo_range[0])
        self.intercept = self.pwm[0] - self.slope * servo_range[0]

#         self.ser = serial.Serial(port, 115200)
#         if not self.ser.isOpen():
#             raise Exception('Arm::init() could not open', port)
#         else:
#             print('Arm opened {} @ 115200'.format(port))

#     def __del__(self):
#         time.sleep(3)
#         self.ser.close()
#         time.sleep(0.01)

    def angle2pwm(self, angle):
        """
        angle: servo angle in radians
        return: servo pwm
        """
        angle *= 180/pi
        pwm = self.slope * angle + self.intercept

        # if self.pwm[0] > pwm < self.pwm[1]:
        # 	print('ERROR: {} out of limits {}'.format(pwm, self.pwm))
        # 	raise Exception('PWM value out of range')
        return int(pwm)

    def send(self, angles, cmdSleep=3, speed=2500):
        """
        Converts an array of angles to the servo board command string.
        ex: '#0 P1500 #1 P1500 #2 P1500 #3 P1500 #4 P1500 T2500\r'

        angles: array of arm angles
        """
        # correct dh frame and servo angles
#         angles[3] += pi/2  # angles - wrist correction
#         angles[0] += pi/2  # base correction

        # cmd = '#0 P1500 #1 P1500 #2 P1500 #3 P1500 #4 P1500 T4000\r'
        cmd = []
        for channel, a in enumerate(angles):
            pwm = self.angle2pwm(a)
            if self.pwm[0] > pwm < self.pwm[1]:
                print('ERROR: servo[{}] PWM{} out of limits {}'.format(channel, self.pwm, pwm))
                raise Exception('PWM value out of range')
            cmd.append('#{} P{}'.format(channel, pwm))
        cmd.append('T{}\r'.format(speed))
        cmd = ' '.join(cmd)

        print('[Move] ---------------------------')
        aa = [x*180/pi for x in angles]  # convert to degrees
        print('  angles: {:6.0f} {:6.0f} {:6.0f} {:6.0f}'.format(*aa[:4]))
        print('  claw: {}'.format('open' if aa[4] == 0 else 'closed'))
        print('  cmd: {}\n'.format(cmd))

#         self.ser.write(cmd)
#         time.sleep(speed/1000)

    def inverse(self, x, y, z, orient, claw=0):
        """
        Calculates inverse kinematics
        given: (x,y,z) in 3D space and claw open/closed
        """
        l1 = 5.75
        l2 = 7.375
        l3 = 3.375

        # check workspace constraints
        if z < 0:
            raise Exception('z in ground')
        elif mag(x, y, z) > (l1 + l2 + l3):
            raise Exception('out of reach {} > {}'.format(mag(x, y, z), (l1 + l2 + l3)))

        # get x-y plane azimuth
        t1 = atan2(y, x)

        # Now, most of the arm operates in the w-z frame
        w = mag(x, y)   # new frame axis
        gamma = atan2(z, w)
        r = mag(z, w)

        c = mag(w-l3*cos(orient), z-l3*sin(orient))

        t3 = cosine_law(l1, l2, c, True)

        d = cosine_law(l2, c, l1)
        e = cosine_law(c, l3, r)
        t4 = pi - d - e

        alpha = cosine_law(l1, c, l2)
        beta = cosine_law(c, r, l3)

        t2 = alpha + beta + gamma

        return [t1, t2, t3, t4, claw*pi/180]

    def calibrate(self):
        pass

    def move_arm_angles(self, angles):
        for a in angles:
            a = [pi/180*x for x in a]
            self.send(a)

    def move_arm_points(self, points):
        for pt in points:
            angles = self.inverse(*pt)
            self.send(angles)

In [10]:
arm = Arm('hi', [900, 2100])

angle 1281


In [20]:
# example
angle = arm.angle2pwm(2)
print('angle', angle)

# 1
angle = arm.angle2pwm(1)
print('angle', angle)

angle 1663
angle 1281


In [21]:
# 2
arm.send([1,2,3,2,1])  # test
arm.send([3,2,1,2,3])  # answer

[Move] ---------------------------
  angles:     57    115    172    115
  claw: closed
  cmd: #0 P1281 #1 P1663 #2 P2045 #3 P1663 #4 P1281 T2500

[Move] ---------------------------
  angles:    172    115     57    115
  claw: closed
  cmd: #0 P2045 #1 P1663 #2 P1281 #3 P1663 #4 P2045 T2500



In [15]:
# equalateral triangle
angle = cosine_law(3,3,3)
print('rads', angle)
print('degs', angle*180/pi)

rads 1.0471975512
degs 60.0


In [14]:
# question 3
angle = cosine_law(2,5,4)
print('rads', angle)
print('degs', angle*180/pi)

rads 0.86321189007
degs 49.4583981265


In [16]:
# given example
angs = arm.inverse(3,3,3,0,0)
print('rads', angs)
angs = map(rad2deg, angs)
print('degrees', angs)

rads [0.7853981633974483, 3.188227078176998, 2.729144874301295, 0.45908220387570275, 0.0]
degrees [45.0, 182.67195570886796, 156.36848297722577, 26.303472731642174, 0.0]


In [18]:
# question 4
angs = arm.inverse(3,4,5,pi/2,0)
print('rads', angs)
angs = map(rad2deg, angs)
print('degrees', angs)

rads [0.9272952180016122, 2.7227631298941644, 2.3535468345770267, 0.3692162953171376, 0.0]
degrees [53.13010235415598, 156.00283595676598, 134.84830050763816, 21.154535449127806, 0.0]
